In [1]:
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir

from aequilibrae.paths import TransitAssignment, TransitClass
from aequilibrae.utils.create_example import create_example
import numpy as np

# Imports for GTFS import
from aequilibrae.transit import Transit

# Imports for SF transit graph construction
from aequilibrae.project.database_connection import database_connection
from aequilibrae.project.database_connection import database_connection


In [2]:
fldr = join(gettempdir(), uuid4().hex)
project = create_example(fldr, "coquimbo")

In [3]:
project.network.build_graphs()
car_graph = project.network.graphs["c"]
car_graph.set_blocked_centroid_flows(False)

/home/ubuntu/.local/lib/python3.10/site-packages/aequilibrae/project/network/network.py:337: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.read_sql(sql, conn).fillna(value=np.nan)


In [4]:
# Preload parameters
period_start = int(6.5 * 60 * 60) # 6:30am in seconds from midnight
period_end = int(8.5 * 60 * 60)   # 8:30am in seconds from midnight

In [5]:
transit_conn = database_connection("transit")

# Get list of trip_id's, whose start time is within the period
select_trip_ids = f"SELECT DISTINCT trip_id FROM trips_schedule WHERE arrival BETWEEN {period_start} AND {period_end}"

# Convert the trip_id's to their corresponding pattern_id's
select_pattern_ids = f"SELECT pattern_id FROM trips WHERE trip_id IN ({select_trip_ids})"
period_patterns = map(lambda x: x[0], transit_conn.execute(select_pattern_ids).fetchall())

# For each pattern id get the corresponding list of link/dir'select_pattern_ids
select_links = lambda pattern: f"SELECT link, dir FROM pattern_mapping WHERE pattern_id = {pattern}"
period_links = lambda pattern: transit_conn.execute(select_links(pattern)).fetchall()

In [6]:
# Create dictionary of link, dirs in the graph
proj_conn = database_connection("project_database")
links = proj_conn.execute("SELECT link_id, direction FROM links").fetchall()
proj_conn.close()

links_dict = dict()
for link, dir in links:
    if dir == 0: # Means both directions
        links_dict[(link, 1)] = 0
        links_dict[(link, -1)] = 0
    else:
        links_dict[(link, dir)] = 0

In [7]:
# Iterate through pattern id's and for each link in the sequence, increment the coresponding dictionary value
for pattern_id in period_patterns:
    pattern_links = period_links(pattern_id)
    for link, d in pattern_links:
        dir = -1 if d == 1 else 1 # Changes direction of 0 to 1, ask Pedro when this will be fixed (is supposed to be +-1)
        links_dict[(link, dir)] += 1

transit_conn.close() # No longer needed